In [1]:
# import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import inspect
import time
import pickle

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import src.simulation as s

%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

# Peer Nomination Network Data


In [2]:
# instantiate simulation class

simulation =  s.Simulation()

/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:24: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.nominationPopulation = p.PeerNominatedDataPopulation('Peer-Nomination data population', self.input_args)
/Users/davidblok/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:25: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.communicationPopulation = p.CommunicationDataPopulation('Communic

In [3]:
# Read tuned parameter combinations (See tuning_nom_network.ipynb)
pars_nomination = pd.read_csv('../output/opt_pars_m_nomination_revised.csv', sep=',', header=0)
pars_nomination

,threshold,ipa,error
0,0.0125,0.0050,7.441040e-08
1,0.0050,0.0400,5.940499e-06
2,0.0125,0.0025,2.856581e-05
3,0.0125,0.0075,3.255180e-05
4,0.0100,0.0075,8.834255e-05
...,...,...,...
95,0.0425,0.0100,1.061154e-02
96,0.0500,0.0025,1.062087e-02
97,0.0325,0.0175,1.070055e-02
98,0.0450,0.0100,1.089975e-02


## 1. Pre-Intervention simulations

In [4]:
# Run pre-intervention for each parameter set
list_pre_results_child_nom = []
list_pre_results_class_nom = []
list_snapshot_pa_nom = []

for index, row in pars_nomination.iterrows():
    start = time.time()
    # run pre-intervention
    pre_results_nom, pre_results_avg_nom, snapshot_pa_nom = simulation.simulate_preinterventions(730, 'nomination', row['threshold'], row['ipa'])

    list_pre_results_child_nom.append(pre_results_nom)
    list_pre_results_class_nom.append(pre_results_avg_nom)
    list_snapshot_pa_nom.append(snapshot_pa_nom)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/nomination/list_pre_results_child.obj', 'wb')
pickle.dump(list_pre_results_child_nom, file)

file = open('../output/simulation/nomination/list_pre_results_class.obj', 'wb')
pickle.dump(list_pre_results_class_nom, file)

file = open('../output/simulation/nomination/list_snapshot_pa.obj', 'wb')
pickle.dump(list_snapshot_pa_nom, file)


0 0.0125 0.005 Time elapsed: 9.473083257675171  seconds
1 0.005 0.04 Time elapsed: 11.947841882705688  seconds
2 0.0125 0.0025 Time elapsed: 9.671881914138794  seconds
3 0.0125 0.0075 Time elapsed: 9.054612874984741  seconds
4 0.01 0.0075 Time elapsed: 8.92451286315918  seconds
5 0.015 0.0025 Time elapsed: 8.838012933731079  seconds
6 0.015 0.005 Time elapsed: 8.698210000991821  seconds
7 0.01 0.005 Time elapsed: 8.431485891342163  seconds
8 0.0125 0.01 Time elapsed: 8.384771823883057  seconds
9 0.015 0.0075 Time elapsed: 8.451297044754028  seconds
10 0.017499999999999998 0.0025 Time elapsed: 8.577908039093018  seconds
11 0.01 0.0025 Time elapsed: 8.490892887115479  seconds
12 0.0075 0.0075 Time elapsed: 8.409265041351318  seconds
13 0.0125 0.0125 Time elapsed: 8.459251880645752  seconds
14 0.017499999999999998 0.005 Time elapsed: 8.471168994903564  seconds
15 0.015 0.01 Time elapsed: 8.484577894210815  seconds
16 0.0025 0.02 Time elapsed: 8.832052946090698  seconds
17 0.0075 0.005 Tim

## 2. Intervention Simulations


In [5]:
effect = np.array([0.05, 0.1, 0.15, 0.17, 0.2])

for effect in effect:
    # Run interventions for each parameter combination
    list_results_child_nom = []
    list_results_class_nom = []
    list_influential_agents_nom = []

    for index, row in pars_nomination.iterrows():
        start = time.time()
        # run intervention
        results_nom, results_avg_nom, agents_per_intervention_nom = simulation.simulate_interventions(730, 'nomination', list_snapshot_pa_nom[index], row['threshold'], row['ipa'], effect, index)

        list_results_child_nom.append(results_nom)
        list_results_class_nom.append(results_avg_nom)
        list_influential_agents_nom.append(agents_per_intervention_nom)

        end = time.time()
        print(effect, index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

    # Save model output to object
    name1 = ('../output/simulation/nomination/list_results_child_'+str(effect)+'.obj')
    file = open(name1, 'wb')
    pickle.dump(list_results_child_nom, file)

    name2 = ('../output/simulation/nomination/list_results_class_'+str(effect)+'.obj')
    file = open(name2, 'wb')
    pickle.dump(list_results_class_nom, file)

    name3 = ('../output/simulation/nomination/list_influential_agents_'+str(effect)+'.obj')
    file = open(name3, 'wb')
    pickle.dump(list_influential_agents_nom, file)


0.05 0 0.0125 0.005 Time elapsed: 47.74705791473389  seconds
0.05 1 0.005 0.04 Time elapsed: 49.07481908798218  seconds
0.05 2 0.0125 0.0025 Time elapsed: 47.73432111740112  seconds
0.05 3 0.0125 0.0075 Time elapsed: 47.829777240753174  seconds
0.05 4 0.01 0.0075 Time elapsed: 54.05740404129028  seconds
0.05 5 0.015 0.0025 Time elapsed: 50.754969120025635  seconds
0.05 6 0.015 0.005 Time elapsed: 49.86608004570007  seconds
0.05 7 0.01 0.005 Time elapsed: 50.446922063827515  seconds
0.05 8 0.0125 0.01 Time elapsed: 49.715439796447754  seconds
0.05 9 0.015 0.0075 Time elapsed: 49.48149299621582  seconds
0.05 10 0.017499999999999998 0.0025 Time elapsed: 50.90358901023865  seconds
0.05 11 0.01 0.0025 Time elapsed: 48.599576234817505  seconds
0.05 12 0.0075 0.0075 Time elapsed: 54.042978048324585  seconds
0.05 13 0.0125 0.0125 Time elapsed: 56.48093891143799  seconds
0.05 14 0.017499999999999998 0.005 Time elapsed: 49.63980579376221  seconds
0.05 15 0.015 0.01 Time elapsed: 48.9385120868682